In [54]:
import os

In [55]:
%pwd

'/home/rav'

In [56]:
os.chdir("../")

In [57]:
%pwd

'/home'

In [58]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [59]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [60]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [61]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [62]:
from pathlib import Path


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = None
        self.train_generator = None
        self.valid_generator = None

    def get_base_model(self):
        """Loads the base model from the path provided in the config."""
        try:
            self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        except Exception as e:
            print(f"Error loading model from {self.config.updated_base_model_path}: {e}")
            raise

    def create_data_generator(self, subset: str, is_training: bool = True):
        """Create ImageDataGenerator for training or validation set."""
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # If it's a validation set, no augmentation needed
        if subset == 'validation':
            datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
            return datagen.flow_from_directory(
                directory=self.config.training_data,
                subset=subset,
                shuffle=False,
                **dataflow_kwargs
            )

        # For training, apply augmentations if configured
        if self.config.params_is_augmentation:
            datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        return datagen.flow_from_directory(
            directory=self.config.training_data,
            subset=subset,
            shuffle=True if is_training else False,
            **dataflow_kwargs
        )

    def train_valid_generator(self):
        """Prepares the training and validation data generators."""
        self.valid_generator = self.create_data_generator(subset="validation", is_training=False)
        self.train_generator = self.create_data_generator(subset="training", is_training=True)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """Saves the model to the specified path."""
        try:
            model.save(path)
            print(f"Model saved to {path}")
        except Exception as e:
            print(f"Error saving model: {e}")
            raise

    def train(self):
        """Train the model with the provided data generators."""
        if self.model is None:
            print("Model is not loaded. Please load the base model first.")
            return

        # Calculate the number of steps per epoch and validation steps
        steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        print(f"Training with {steps_per_epoch} steps per epoch and {validation_steps} validation steps.")

        try:
            self.model.fit(
                self.train_generator,
                epochs=self.config.params_epochs,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps,
                validation_data=self.valid_generator
            )
            # Save model after training
            self.save_model(path=self.config.trained_model_path, model=self.model)
        except Exception as e:
            print(f"Error during training: {e}")
            raise


In [63]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config/config.yaml'